# Introduction

After creating the Tapis pod for a Neo4j database, we need to add data to it. It is very easy to load in CSV data.

In [1]:
import py2neo
import pandas
from py2neo import Graph, Node, Relationship
from py2neo import GraphService
import time
from getpass import getpass
from tapipy.tapis import Tapis

import requests
import io

In [82]:
start = time.time()

# Base URL for Tapis
base_url = "https://icicle.develop.tapis.io"
username = str(input("username"))

# Get Tapis object if it isn't already created.
try:
    t
    if t.base_url == base_url and t.username == username and t.access_token:
        print("Tapis object already exists.")
        if t.access_token.expires_at < datetime.datetime.now(pytz.utc):
            print("Existing Tapis token expired, getting new token.")
            raise
    else:
        print("Creating new Tapis object.")
        raise
except:
    try:
        t = Tapis(base_url = base_url,
                  username = username,
                  password = getpass('password'))
        t.get_tokens()
    except Exception as e:
        print(f"\nBROKEN! timeout: {time.time() - start}\n")
        raise

# V3 Headers
header_dat = {"X-Tapis-token": t.access_token.access_token,
              "Content-Type": "application/json"}

# Service URL
url = f"{base_url}/v3"                   # remote

print(time.time() - start)
print(f"base_url: {base_url}")
print(f"serv_url: {url}")

8.316059589385986
base_url: https://icicle.develop.tapis.io
serv_url: https://icicle.develop.tapis.io/v3


In [85]:
t.pods.get_pods()

[
 creation_ts: None
 data_attached: []
 data_requests: []
 description: sample description
 environment_variables: 
 
 pod_id: michaelraytesting
 pod_template: neo4j
 roles_inherited: []
 roles_required: []
 status: RUNNING
 status_container: 
 message: Pod is running.
 phase: Running
 start_time: 2022-07-29 22:31:37+00:00
 status_requested: ON
 update_ts: None
 url: michaelraytesting.pods.icicle.develop.tapis.io,
 
 creation_ts: None
 data_attached: []
 data_requests: []
 description: The pod for the REHS final KG
 environment_variables: 
 
 pod_id: finalkg
 pod_template: neo4j
 roles_inherited: []
 roles_required: []
 status: RUNNING
 status_container: 
 message: Pod is running.
 phase: Running
 start_time: 2022-07-29 20:20:36+00:00
 status_requested: ON
 update_ts: None
 url: finalkg.pods.icicle.develop.tapis.io,
 
 creation_ts: None
 data_attached: []
 data_requests: []
 description: The second pod for the REHS final project.
 environment_variables: 
 
 pod_id: finalkg2
 pod_templ

In [86]:
# See pod ID using above jupyter cell
pod_id = str(input("Enter a pod ID. ")).lower()
username, password = t.pods.get_pod_credentials(pod_id=pod_id).user_username, t.pods.get_pod_credentials(pod_id=pod_id).user_password

In [167]:
# Connection to neo4j database
graph = Graph(f"bolt+ssc://{pod_id}.pods.icicle.develop.tapis.io:443", auth=(username, password), secure=True, verify=True)

# How does it work?
We first call LOAD CSV WITH HEADERS and then give it a url which contains the CSV file to load.

Merge associates a set of nodes with labels to a column or columns of data. For example, in the below code, nodes with label PLATFORM are created which contain data from the row with header Platform. Similarly, nodes with label GAME contain the specified data from the rows.

We can also create relationships upon load. The below example simply creates a relationship from every game to the platform it is associated with.

In [192]:
dataset_link = r'https://raw.githubusercontent.com/sdsc-hpc-students/REHS2022/main/Final-Project/ExtraKGs/celesital-bodies/processed_data.csv'

In [193]:
csv_file = requests.get(dataset_link).content       
csv_columns = pandas.read_csv(io.StringIO(csv_file.decode('utf-8')))
csv_columns.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous,predicted_values,Classification_Success
0,54203701,(2021 TQ3),0.010105,0.022596,26091.984595,5.043360e+07,Earth,0.0,27.10,0.0,0.0,Correctly_Classified
1,3656413,(2014 BP),0.025384,0.056760,36084.475960,3.945816e+06,Earth,0.0,25.10,0.0,1.0,Incorrectly_Classified
2,54017120,(2020 KP1),0.063761,0.142574,61507.693535,2.525034e+07,Earth,0.0,23.10,0.0,0.0,Correctly_Classified
3,54225510,(2021 WV4),0.116026,0.259442,32676.917900,6.970577e+07,Earth,0.0,21.80,0.0,0.0,Correctly_Classified
4,3183851,(2004 LY5),0.122618,0.274183,103279.252357,9.670783e+06,Earth,0.0,21.68,1.0,0.0,Incorrectly_Classified


In [194]:
def get_columns(csv_link):
    csv_file = requests.get(csv_link).content       
    csv_columns = pandas.read_csv(io.StringIO(csv_file.decode('utf-8'))).columns.tolist()

    return csv_columns

In [195]:
columns = get_columns(dataset_link)
columns

['id',
 'name',
 'est_diameter_min',
 'est_diameter_max',
 'relative_velocity',
 'miss_distance',
 'orbiting_body',
 'sentry_object',
 'absolute_magnitude',
 'hazardous',
 'predicted_values',
 'Classification_Success']

In [202]:
hazardous = {'node_type':'hazardous', 'properties':None, 'relationships':None}
predicted_values = {'node_type':'predicted_values', 'properties':None, 'relationships':None}
Classifications_Success = {'node_type':'Classifications_Success', 'properties':None, 'relationships':None}
asteroid = {'node_type':'asteroid',
            'properties':['id', 'name', 'est_diameter_min', 'est_diameter_max', 'relative_velocity', 'miss_distance', 'sentry_object', 'absolute_magnitude'], 
            'relationships':{'is':hazardous, 'is_in_group':predicted_values}}#,'prediction_was':Classifications_Success}}

nodes = [asteroid, hazardous, predicted_values]#, Classifications_Success]

In [161]:
def make_properties(node):
    properties = ''
    if node['properties']:
        for property_name in node['properties']:
            properties += f'{property_name}:row.{property_name}, '
        return properties[:-2]
    else:
        properties += f'id: row.{node["node_type"]}'
        return properties

In [162]:
def make_relationships(nodes, node):
    relationships = ''
    for relationships_type, related_to in node['relationships'].items():
        relationships += f'\nMERGE (n{nodes.index(node)})-[:{relationships_type}]->(n{nodes.index(related_to)})'

    return relationships

In [169]:
def expression_maker(nodes, csv_link=None):
    script = f'LOAD CSV WITH HEADERS FROM "{csv_link}" AS row WITH row WHERE row.id IS NOT null'
    for index, node in enumerate(nodes):
        properties = make_properties(node)
        script += f'\nMERGE (n{index}:{node["node_type"]}' + ' {' + properties + '})'

    for node in nodes:
        if node['relationships']:
            relationships = make_relationships(nodes, node)
            script += relationships

    return script

In [203]:
expression = expression_maker(nodes, dataset_link)

In [204]:
print(expression)

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/sdsc-hpc-students/REHS2022/main/Final-Project/ExtraKGs/celesital-bodies/processed_data.csv" AS row WITH row WHERE row.id IS NOT null
MERGE (n0:asteroid {id:row.id, name:row.name, est_diameter_min:row.est_diameter_min, est_diameter_max:row.est_diameter_max, relative_velocity:row.relative_velocity, miss_distance:row.miss_distance, sentry_object:row.sentry_object, absolute_magnitude:row.absolute_magnitude})
MERGE (n1:hazardous {id: row.hazardous})
MERGE (n2:predicted_values {id: row.predicted_values})
MERGE (n0)-[:is]->(n1)
MERGE (n0)-[:is_in_group]->(n2)


In [205]:
flag = True

while flag:
    try:
        graph.run(expression).to_data_frame()
        flag = False
    except py2neo.ConnectionBroken:
        print('Failed to connect\nreconnecting now...')
        graph = Graph(f"bolt+ssc://{pod_id}.pods.icicle.develop.tapis.io:443", auth=(username, password), secure=True, verify=True)
        continue

In [173]:
# Loading CSV Example
script = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/sdsc-hpc-students/REHS2022/main/Final-Project/ExtraKGs/vgsales.csv" AS row
MERGE (n:PLATFORM {{id:row.Platform}})
MERGE (n1:GAME {{id:row.Name, rank:row.Rank, year:row.Year, genre:row.Genre, 
publisher:row.Publisher, NA_Sales:row.NA_Sales, EU_SALES:row.EU_Sales,JP_Sales:row.JP_Sales, 
OTHER_Sales:row.Other_Sales}})
MERGE (n1)-[:PART_OF]->(n)
"""
graph.run(script).to_data_frame()

ClientError: [Statement.SyntaxError] Invalid input '{': expected an identifier (line 3, column 20 (offset: 157))
"MERGE (n:PLATFORM {{id:row.Platform}})"
                    ^